In [199]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from search_engine import FaissSearchEngine
from typing import List, Dict, Optional
import pandas as pd


In [198]:
!ollama pull gemma3:12b

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling adca500fad9b...   0% ▕                ▏    0 B/8.1 GB                  pulling manifest 
pulling adca500fad9b...   0% ▕                ▏    0 B/8.1 GB                  pulling manifest 
pulling adca500fad9b...   0% ▕                ▏    0 B/8.1 GB                  pulling manifest 
pulling adca500fad9b...   0% ▕                ▏    0 B/8.1 GB                  pulling manifest 
pulling adca500fad9b...   0% ▕                ▏    0 B/8.1 GB                  pulling manifest 
pulling adca500fad9b...   0% ▕                ▏ 1.6 MB/8.1 GB                  pulling manifest 
pulling adca500fad9b...   0% ▕                ▏ 2.4 MB/8.1 GB                  pulling manifest 
pulling adca500fad9b...   0% ▕                ▏ 3.5 MB/8.1 GB                  pulling manifes

In [200]:
items_bd = pd.read_csv('../../Downloads/items_bd_final.csv')
items_bd.head()

,Unnamed: 0,country_code,city_code,store_name,product_name,collection_section,product_description,subset,hash,aux_store,HIER,s3_path,full_name,id,Персональный_кэшбек,Заказов_за_месяц,Персональный_кэшбэк,Цена,Рейтинг,Время_доставки
0,753,UA,MPL,Mikado,Рол Кампай (220 г),РОЛИ DELUX,"Угорь, лосось, том ям, огурец, лист салата, сы...",train,3903697417650224690,False,True,dataset/BDKZJSR_0000806_2000078286.png,"Рол Кампай (220 г) \n Угорь, лосось, том ям, о...",25497,20%,0,0%,1730₽,4.71,40 мин
1,5766,UA,MPL,New York Street Pizza,Тотори Сикору (220 г),Эти роли,"Морской гребешок, сыр для суши, огурец, икра к...",train,-3451380953300451258,False,True,dataset/BDKZJSR_0000503_1927787986.png,"Тотори Сикору (220 г) \n Морской гребешок, сыр...",85908,0%,0,5%,1790₽,4.85,30 мин
2,6601,UA,MPL,New York Street Pizza,Салат Цезарь с лососем (200 г),Салаты и антипасты,"Салат из разных овощей, слабосоленый лосось, п...",train,-3276916747514226914,False,True,dataset/BDKZJSR_0000423_1927638127.png,Салат Цезарь с лососем (200 г) \n Салат из раз...,12266,0%,0,0%,1490₽,4.89,40 мин
3,7153,UA,MPL,Detroit. City diner,Рыба и чипсы (280 г),На старт!,С чесночным йогуртом,train,-991058856461244767,False,True,dataset/BDKZJSR_0000538_1944956442.png,Рыба и чипсы (280 г) \n С чесночным йогуртом,53171,5%,1,0%,740₽,4.92,20 мин
4,7771,UA,MPL,Mafia / Мафія,"Газировка «Моршинська» (0,75 л)",Напитки,Минеральная газированная вода,train,-8606000751944571868,False,True,dataset/BDKZJSR_0000103_1882646104.png,"Газировка «Моршинська» (0,75 л) \n Минеральная...",20355,5%,0,0%,1060₽,4.59,20 мин


In [201]:
MODEL_PATH = '../models/rubert-tiny-turbo'
INDEX_PATH = 'faiss_index.bin'

search_engine = FaissSearchEngine(model_path=MODEL_PATH)
search_engine.load_index(INDEX_PATH)

Index and data loaded successfully


In [202]:
items_bd.iloc[0]['id']

np.int64(25497)

In [203]:
!pip list | grep ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


langchain-ollama         0.2.3
ollama                   0.4.7


In [221]:
class ShoppingAssistant:
    def __init__(self, model, items_bd: pd.DataFrame, search_engine: FaissSearchEngine, path_to_user_description:str):
        self.llm = model
        self.items_bd = items_bd
        self.search_engine = search_engine

        # истории диалога
        self.memory = ConversationBufferMemory() 
        self.readonly_memory = ReadOnlySharedMemory(memory=self.memory)
        
        # id текущих товаров в корзине пользователя
        self.current_product_ids: List[int] = []

        # id обновленных товаров в корзине пользователя
        self.updated_product_ids: List[int] = []

        # инициализация агентуры
        self._initialize_agents()

        # подргузка суммаризированной информации о пользователе
        self._load_user_description(path_to_user_description)
    

    def _load_user_description(self, path_to_user_description: str) -> str:
        with open(path_to_user_description, 'r', encoding='utf-8') as file:
            self.user_description = file.read()

    
    def _initialize_agents(self):
        """Инициализация агентуры"""
        self.category_agent = self._create_category_agent()
        self.editor_agent = self._create_editor_agent()
        self.reasoning_agent = self._create_reasoning_agent()
        self.best_item_agent = self._create_best_item_agent()
        self.explainer_agent = self._create_explainer_agent()
    
    def _create_category_agent(self) -> LLMChain:
        """Агент по подсказке категорий товаров / конкретных блюд для поиска по базе данных.
        Пример:
            U - Хочу колу и какую нибудь пасту
            A - Coca cola, Паста
            ------
            U - Закажи еду для киновечеринки на троих
            A - Пицца комбо на троих, Газированные напитки, Снэки
        """
        return LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["history", "input"],
                template="""Ты помощник по подбору еды. Твоя задача — предложить категории продуктов, которые максимально соответствуют запросу пользователя. 
                **Правила:**
                1. Учитывай количество человек, количество блюд и конкретные запросы.
                - Если указано количество (например, "3 пиццы"), предложи 3 разных пиццы.
                - Если количество не указано - предложи 1 категорию.
                - Если упоминается конкретный продукт (например, "кока-кола"), добавь его в список.
                - Если запрос общий (например, "что-нибудь сладкое"), предложи несколько категорий на свое усмотрение.
                2. Стремись к разнообразию: предлагай разные блюда или напитки, если запрос позволяет.
                3. Используй историю диалога для учета предпочтений пользователя.
                
                *Примеры:**
                Запрос: "3 пиццы и что-то сладкое"
                Ответ: "Маргарита, Пепперони, Четыре сыра, Тирамису, Эклеры"

                Запрос: "Кока колу и чипсы"
                Ответ: "Кока кола, чипсы"

                Запрос: "Закажи 2 бутылки колы и лапшу"
                Ответ: "Кола, кола, лапша"

                Запрос: "маргарита и рол":
                Ответ: "Пицца маргарита, рол"

                Запрос: "чипсы покорн пиццу и лимонад"
                Ответ: "Чипсы, Попкорн, пицца, лимонад"

                Запрос: "Что нибудь веганское и газировку"
                Ответ: "Веганское, газировка"

                Запрос: "какой нибудь ролл"
                Ответ: "Ролл"

                **Контекст диалога:**
                История: {history}
                
                **Запрос пользователя:** {input}
                
            **Категории (только названия через запятую):**"""),
            memory=self.readonly_memory,
            output_key="categories"
        )
    
    def _create_explainer_agent(self) -> LLMChain:
        """Объясняет пользователю, на основе чего был сделан конкретный выбор продуктов в его корзине."""
        return LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["best_products", "input", "history"],
                template="""Ты очень доброжелательный и дружелюбный ассистент, помогающий пользователю с выбором продуктов.
                Тебе будет дан список товаров и запрос пользователя.
                Объясни пользователю, почему товары из спискаподходят ему лучше всего исходя из его запроса в 2-3 предложениях.
                **Правила**
                    1 - Начни похожей с фразы: "Нашел для вас подходящие товары!"
                    2 - Обязательно аргументируй, почему эти товары подходят лучше всего, например: 
                        - На товар X у вас персональный кешбек 5%!
                        - Товар X будет доставлен быстрее всех похожих!
                        - Цена на этот товар X рублей - что дешевле аналогичных товаров в других магазинах!
                        - Этот товар обладает высоким рейтингом - X!
                        - Ранее вы уже заказывали этот товар / похожий товар!
                    3 - Закончи предложение так: "Хотите оформить заказ?")
                Список товаров: {best_products}
                История диалога:
                {history}
                Запрос: {input}
                
                Твой ответ (1-2 предложения):"""

            ),
            memory=self.readonly_memory,
            output_key="explanation"
        )
    
    def _create_best_item_agent(self) -> LLMChain:
         """Выбирает лучший товар из категории на основе запроса пользователя, его истории и характеристик товара."""
         return LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["relevant_products", "input", "history", "user_description", "current_product"],
                template="""Тебе будет дан список товаров, суммаризованные предпочтения пользователя, запрос пользователя и конкретная категория.
                Выбери ОДИН лучший товар на основе запроса пользователя и выведи его id.
                Выводи только id лучшего товара и больше ничего.
                В первую очередь смотри за текущий запрос пользователя и на конкретную категорию, потом на его предпочтения.
                Выбирай товары, похожие по названию на те, что ищет пользователь.
                **Правила выбора:**
                    1. Приоритеты пользователя (по порядку):
                    - Цена (дешевле > дороже)
                    - Скорость доставки (быстрее > медленнее)
                    - Состав (точное соответствие требованиям)
                    - Кэшбек (выше процент > ниже)
                    2. Если точные критерии не указаны — используй контекст общения с пользователем и рейтинг товара.
                    3. Формат ответа ТОЛЬКО числовой ID без пояснений

                Суммаризованные предпочтения пользователя:
                {user_description}

                История диалога:
                {history}

                Конкретная категория:
                {current_product}

                Список товаров: {relevant_products}
                
                Запрос: {input}
                id наиболее подходящего под запрос товара:
                """
            ),
            memory=self.readonly_memory,
            output_key="best_product_id"
        )


    def _create_reasoning_agent(self) -> LLMChain:
        """Создает цепочку рассужддений для обоснования пользователю на основе каких критериев и пожеланий идет поиск по базе данных."""
        return  LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["input", "history"],
                template="""Ты ассистент анализа запросов. Сгенерируй 3 ключевых направления поиска в формате временных шагов, 
                которые будут отображаться пользователю во время обработки запроса. 

                **Правила генерации:**
                1. Каждый шаг должен начинаться с глагола действия (Ищу/Анализирую/Сравниваю)
                2. Использовать максимум 8 слов на шаг
                3. Добавлять эмодзи 
                4. Каждое направление выводи с новой строки
                5. Выводи ТОЛЬКО направления без дополнительной информации

                **Примеры:**
                🔄 Анализирую историю заказов (3s)...
                💰 Сравниваю цены...
                🕒 Ищу доставку быстрее 30 минут...
                🌱 Анализирую ваши вкусовые предпочтения...
                💰 Ищу на несколько человек...
                🌱 Учитываю веганские предпочтения...

                **Текущий контекст:**
                История диалога: {history}
                Запрос: {input}

                **Формат вывода:**
                [Шаг 1 с эмодзи] 
                [Шаг 2 с эмодзи]
                [Шаг 3 с эмодзи]"""
            ),
            memory=self.readonly_memory,
            output_key="reasoning"
        ) 
    
    def _create_editor_agent(self) -> LLMChain:
        """Редактирует корзину пользователя исходя из его запроса.
        Пример:
            curent_items: [Coca cola, Пицаа маргарита]
            
            U - поменяй на пиццу на какую нибудь без мяса
            A - [Coca cola, Пицца веганская]
        """
        return LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["current_products", "input", "history"],
                template="""Тебе будет дан список товаров и комментарий пользователя. Отредактируй список товаров на основе комментария пользователя.
                Если товар менять не надо, выведи его название без изменений.
                Если пользователь хочет что-то другое, замени товар.
                Если пользователь просит что-то добавить, добавь новый товар к текущим товарам.
                Если пользователь прочит удалить товар, удали конкретный товар.

                Примеры взаимодействия:
                    Текущие товары: [пицца, кола]
                    Запрос: добавь чего-нибудь сладкого
                    Обновленный список товаров: [пицца \n кола \n торт]

                    Текущие товары: [Coca cola \n пирожки]
                    Запрос: замени кока колу на другую газировку
                    Обновленный список товаров: [Sprite \n пирожки]

                    Текущие товары: [Рол сырный \n пирожки]
                    Запрос: поменяй сырный рол на мясной
                    Обновленный список товаров: [рол мясной \n пирожки]

                    Текущие товары: [Огурцы]
                    Запрос: Добавь чего нибудь остренького мексиканского
                    Обновленный список товаров: [огурцы \n сальса \n тако]

                    Текущие товары: [сникерс \n спрайт]
                    Запрос: удали спрайт
                    Обновленный список товаров: [сникерс]
                
                История диалога:
                {history}

                Текущие товары:
                {current_products}
                
                Запрос: {input}
                Обновленный список товаров (только названия, каждое с новой строки):"""
            ),
            memory=ConversationBufferMemory(
                memory_key="history",
                input_key="input"  # Явное указание ключа для памяти
            ),
            output_key="updated_products"
        )


    def get_relevant_pruduct_id(self, user_input: str, product: str) -> str:
        relevant_products = self.search_engine.search(product, top_n=10)
        relevant_products =  ' \n '.join(list(map(str, relevant_products)))

        best_product = self.best_item_agent({
            "input": user_input,
            "history": self.readonly_memory.load_memory_variables({})["history"],
            "relevant_products": relevant_products,
            "current_product": product,
            "user_description": self.user_description
        })

        return best_product['best_product_id'].strip()
    
    def get_product_name_by_id(self, ids: List[int]) -> List[int]:
        """Возвращает названия товаров по их ID."""
        return list(self.items_bd[self.items_bd['id'].isin(ids)]['product_name'].values)
    
    def get_full_name_by_id(self, ids: List[int]) -> List[int]:
        """Возвращает названиt и описание товара по их ID."""
        return list(self.items_bd[self.items_bd['id'].isin(ids)]['full_name'].values)
    
    def get_current_product_ids(self) -> List[int]:
        return self.current_product_ids


    def get_product_description_for_explainer(self, product_ids):
        """Формирует описание товаров для объяснения выбора."""
        product_descriptions = []
        for id in product_ids:
            sample = self.items_bd[self.items_bd['id'] == id].iloc[0]
            product_descriptions.append({
                    'full_name': sample['full_name'],
                    'id': int(sample['id']),
                    'Цена': sample['Цена'],
                    'Заказов_за_месяц_у_пользователя': int(sample['Заказов_за_месяц']),
                    'Персональный_кэшбэк': sample['Персональный_кэшбэк'],
                    'Рейтинг': str(sample['Рейтинг'])

                })
        return product_descriptions
    
    def get_reasoing_response(self, user_input: str) -> List[str]:
        reasoning = self.reasoning_agent({
            "input": user_input,
            "history": self.readonly_memory.load_memory_variables({})["history"]
        })
        reasoning_points = reasoning["reasoning"].split("\n")

        reasoning_points = [stripped for stripped in (reasoning_point.strip() for reasoning_point in reasoning_points) if len(stripped) >= 5]

        return reasoning_points
    

    def get_explanation_response(self, user_input: str) -> str:
        explanation = self.explainer_agent({
            "input": user_input,
            "history": self.readonly_memory.load_memory_variables({})["history"],
            "best_products": self.get_product_description_for_explainer(self.updated_product_ids)
        })


        return explanation['explanation']

    

    def process_initial_message(self, user_input: str) -> None:
        """Обрабатывает первоначальный запрос пользователя."""

        response = self.category_agent({
            "input": user_input,
            "history": self.readonly_memory.load_memory_variables({})["history"]
        })
        print(response)

        generated_products = response["categories"].split(", ")
        best_product_ids = []

        for product in generated_products:
            if len(product) <= 3:
                continue
            relevant_product_id = self.get_relevant_pruduct_id(user_input, product)

            try:
                best_product_id = int(relevant_product_id)
                best_product_ids.append(best_product_id)
            except:
                continue

        self.current_product_ids = list(self.items_bd[self.items_bd['id'].isin(best_product_ids)]['id'].values)
        self.current_product_ids = [int(product_id) for product_id in self.current_product_ids]

        self.updated_product_ids = self.current_product_ids

        return None
    

    def edit_bascket(self, user_input:str) -> None:

        response = self.editor_agent({
            "input": user_input,  # Основной ключ для памяти
            "current_products": ", ".join(self.get_product_name_by_id(self.current_product_ids)),
            "history": self.readonly_memory.load_memory_variables({})["history"]
        })

        print('Текущие товары')
        print(self.get_product_name_by_id(self.current_product_ids))
        print('Обновленные товары')
        print(response['updated_products'])
        updated_products = [p.strip() for p in response["updated_products"].split("\n")]
        new_products = list(set(updated_products) - set(self.get_product_name_by_id(self.current_product_ids)))

        id2name = {}
        for product_id in self.current_product_ids:
            name = self.get_product_name_by_id([product_id])
            if name:
                id2name[product_id] = name[0]

        updated_product_ids = []
        for product_id, product_name in id2name.items():
            if product_name in updated_products:
                updated_product_ids.append(product_id)


        new_product_ids = []

        for product in new_products:
            print('Новый продукт')
            print(product)
            if len(product) <= 3:
                continue
            relevant_product_id = self.get_relevant_pruduct_id(user_input, product)

            try:
                best_product_id = int(relevant_product_id)
                updated_product_ids.append(best_product_id)
                new_product_ids.append(best_product_id)
            except:
                continue

        self.updated_product_ids = new_product_ids

        current_product_ids = [int(product_id) for product_id in updated_product_ids]
        self.current_product_ids = current_product_ids

        return None
        

    def process_message(self, user_input:str) -> None:
        """Основной метод обработки сообщений пользователя."""
        if not self.current_product_ids:
            self.process_initial_message(user_input)
        else:
            self.edit_bascket(user_input)
        
        return None

In [222]:
model = OllamaLLM(model="gemma2")
path_to_user_description = 'user_description.txt'

In [223]:
items_bd[items_bd['id'] == 25497].iloc[0]['full_name']

'Рол Кампай (220 г) \n Угорь, лосось, том ям, огурец, лист салата, сыр чеддер, кунжут, унаги соус, рис, нори'

In [224]:
assistant = ShoppingAssistant(model=model, items_bd=items_bd, search_engine=search_engine, path_to_user_description=path_to_user_description)


user_input = "Закажи что нибудь под киновечер"

reasoning = assistant.get_reasoing_response(user_input)
print('Поиск:', reasoning)

assistant.process_message(user_input)
print('Найденные товары:\n', assistant.get_full_name_by_id(assistant.current_product_ids))

explanation = assistant.get_explanation_response(user_input)
print('Аргументация выбранных товаров:\n', explanation)






Поиск: ['🍿 Ищу фильмы по рейтингу 🤔', '🎞️ Анализирую вашу подборку 🎬', '🍕 Сравниваю рестораны доставки 🥢']
{'input': 'Закажи что нибудь под киновечер', 'history': '', 'categories': 'Попкорн, пицца, наггетсы, пиво, газировка  \n'}
Найденные товары:
 ['Рол Арігато (230 г) \n Рис, нори, слабосоленый лосось, угорь, креветки, сыр Филадельфия, кунжут, чика, соус унаги.', 'Вишневое пиво «Специальное Ром» (500 мл) \n Пивной коктейль с приятным тонизирующим вкусом романовой вишни, легкий, бодрящий, освежающий, созданный по мотивам европейской трендовой культуры создания коктейлей на основе пива.', 'Мясная пицца \n Моцарелла, куриная грудка, ветчина, помидоры, шампиньоны, томатный соус.', 'Миринда (0,5 л) \n Газированный напиток', 'Пицца Пеперони \n Сыр моцарелла, оливки, пепперони, томатный соус']
Аргументация выбранных товаров:
 Нашел для вас подходящие товары! Пицца Пеперони и Мясная пицца -  идеальный выбор для киносеанса, они вкусные, сытные и отлично сочетаются с любимым фильмом. Хотите оф

In [225]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Рол Арігато (230 г) \n Рис, нори, слабосоленый лосось, угорь, креветки, сыр Филадельфия, кунжут, чика, соус унаги.',
 'Вишневое пиво «Специальное Ром» (500 мл) \n Пивной коктейль с приятным тонизирующим вкусом романовой вишни, легкий, бодрящий, освежающий, созданный по мотивам европейской трендовой культуры создания коктейлей на основе пива.',
 'Мясная пицца \n Моцарелла, куриная грудка, ветчина, помидоры, шампиньоны, томатный соус.',
 'Миринда (0,5 л) \n Газированный напиток',
 'Пицца Пеперони \n Сыр моцарелла, оливки, пепперони, томатный соус']

In [226]:
user_input = "давай вместо Пеперони какую нибудь другую пиццу"

reasoning = assistant.get_reasoing_response(user_input)
print('Поиск:', reasoning)

assistant.process_message(user_input)
print('Найденные товары:\n', assistant.get_full_name_by_id(assistant.current_product_ids))

explanation = assistant.get_explanation_response(user_input)
print('Аргументация выбранных товаров:\n', explanation)

Поиск: ['🍕 Ищу альтернативы пеперони  🍕', '🤔 Анализирую ваши вкусы 🤔', '🍝 Сравниваю доступные варианты 🍝']
Текущие товары
['Рол Арігато (230 г)', 'Вишневое пиво «Специальное Ром» (500 мл)', 'Мясная пицца', 'Миринда (0,5 л)', 'Пицца Пеперони']
Обновленные товары
Рол Арігато (230 г)
Вишневое пиво «Специальное Ром» (500 мл)
Мясная пицца 
Миринда (0,5 л)
Маргарита  



Новый продукт

Новый продукт
Маргарита
Найденные товары:
 ['Рол Арігато (230 г) \n Рис, нори, слабосоленый лосось, угорь, креветки, сыр Филадельфия, кунжут, чика, соус унаги.', 'Вишневое пиво «Специальное Ром» (500 мл) \n Пивной коктейль с приятным тонизирующим вкусом романовой вишни, легкий, бодрящий, освежающий, созданный по мотивам европейской трендовой культуры создания коктейлей на основе пива.', 'Мясная пицца \n Моцарелла, куриная грудка, ветчина, помидоры, шампиньоны, томатный соус.', 'Миринда (0,5 л) \n Газированный напиток', 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.']
Аргуме

In [227]:
assistant.get_full_name_by_id(assistant.current_product_ids)


['Рол Арігато (230 г) \n Рис, нори, слабосоленый лосось, угорь, креветки, сыр Филадельфия, кунжут, чика, соус унаги.',
 'Вишневое пиво «Специальное Ром» (500 мл) \n Пивной коктейль с приятным тонизирующим вкусом романовой вишни, легкий, бодрящий, освежающий, созданный по мотивам европейской трендовой культуры создания коктейлей на основе пива.',
 'Мясная пицца \n Моцарелла, куриная грудка, ветчина, помидоры, шампиньоны, томатный соус.',
 'Миринда (0,5 л) \n Газированный напиток',
 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.']

In [228]:
user_input = "замени вишневое пиво на лимонадик"

reasoning = assistant.get_reasoing_response(user_input)
print('Поиск:', reasoning)

assistant.process_message(user_input)
print('Найденные товары:\n', assistant.get_full_name_by_id(assistant.current_product_ids))

explanation = assistant.get_explanation_response(user_input)
print('Аргументация выбранных товаров:\n', explanation)

Поиск: ['🔄 Ищу лимонадики  🍋', '💰 Сравниваю цены 🤔', '📦  Анализирую варианты доставки 🚚']
Текущие товары
['Рол Арігато (230 г)', 'Вишневое пиво «Специальное Ром» (500 мл)', 'Мясная пицца', 'Миринда (0,5 л)', 'Пицца Маргарита (400 г)']
Обновленные товары
Рол Арігато (230 г)
Лимонадик 
Мясная пицца
Миринда (0,5 л)
Пицца Маргарита (400 г)  

Новый продукт

Новый продукт
Лимонадик
Найденные товары:
 ['Рол Арігато (230 г) \n Рис, нори, слабосоленый лосось, угорь, креветки, сыр Филадельфия, кунжут, чика, соус унаги.', 'Мясная пицца \n Моцарелла, куриная грудка, ветчина, помидоры, шампиньоны, томатный соус.', 'Миринда (0,5 л) \n Газированный напиток', 'Лимонад "Смородина-Ментол" \n Безалкогольный напиток', 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.']
Аргументация выбранных товаров:
 Нашел для вас подходящие товары! Лимонад "Смородина-Ментол" - это отличный выбор вместо вишневого пива, так как он освежающий и без алкоголя. Хотите оформить заказ? 



In [229]:
assistant.get_full_name_by_id(assistant.current_product_ids)


['Рол Арігато (230 г) \n Рис, нори, слабосоленый лосось, угорь, креветки, сыр Филадельфия, кунжут, чика, соус унаги.',
 'Мясная пицца \n Моцарелла, куриная грудка, ветчина, помидоры, шампиньоны, томатный соус.',
 'Миринда (0,5 л) \n Газированный напиток',
 'Лимонад "Смородина-Ментол" \n Безалкогольный напиток',
 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.']

In [186]:
user_input = "Добавь еще снеков"

reasoning = assistant.get_reasoing_response(user_input)
print('Поиск:', reasoning)

assistant.process_message(user_input)
print('Найденные товары:\n', assistant.get_full_name_by_id(assistant.current_product_ids))

explanation = assistant.get_explanation_response(user_input)
print('Аргументация выбранных товаров:\n', explanation)

Поиск: ['🔎 Ищу популярные снеки для заказа', '🍕 Сравниваю варианты с доставкой', '😋 Учитываю ваши любимые вкусы']
Текущие товары
['Пицца Маргарита (400 г)', 'Coca-Cola (1 л)']
Обновленные товары
Пицца Маргарита (400 г)
Coca-Cola (1 л)
Чипсы
Печенье

Новый продукт

Новый продукт
Печенье
Новый продукт
Чипсы
Найденные товары:
 ['Святослав Пече Neill - безумец (970 г.) \n Камикадзе, ронин, печеный дракон', 'Картофельные чипсы куриные кари (50 г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических добавок вкуса.', 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.', 'Coca-Cola (1 л) \n Сладкий газированный напиток']
Аргументация выбранных товаров:
 Нашел для вас подходящие товары! У нас есть "Святослав Пече Neill - безумец (970 г.) Камикадзе, ронин, печеный дракон" с персональным кэшбэк

In [187]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Святослав Пече Neill - безумец (970 г.) \n Камикадзе, ронин, печеный дракон',
 'Картофельные чипсы куриные кари (50 г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических добавок вкуса.',
 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.',
 'Coca-Cola (1 л) \n Сладкий газированный напиток']

In [188]:
assistant.process_message("Добавь чего нибудь остренького мексиканского")

Текущие товары
['Святослав Пече Neill - безумец (970 г.)', 'Картофельные чипсы куриные кари (50 г)', 'Пицца Маргарита (400 г)', 'Coca-Cola (1 л)']
Обновленные товары
Святослав Пече Neill - безумец (970 г.)
Картофельные чипсы куриные кари (50 г)
Пицца Маргарита (400 г)
Coca-Cola (1 л)
Сальса
Тако

Новый продукт

Новый продукт
Тако
Новый продукт
Сальса


In [189]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Святослав Пече Neill - безумец (970 г.) \n Камикадзе, ронин, печеный дракон',
 'Рол "Зеленый Дракон" (8 шт./250 г) \n Нори, рис, угорь, огурец, томаго, икра масаго, авокадо, унаги соус, кунжут.',
 'Картофельные чипсы куриные кари (50 г) \n Филе курицы, маринованное с пряными специями, высушенное в специальных печах с добавлением горчицы. В отличие от обычных чипсов не содержит консервантов, большого количества масла, модифицированного крахмала и синтетических добавок вкуса.',
 'Салат из лосося (250 г) \n Филе лосося слабосоленого, киноа, сыр филадельфия, авокадо, бобы эдамаме, помидоры черри, микс салата, заправка для салата, огурец, начос, кунжут',
 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.',
 'Coca-Cola (1 л) \n Сладкий газированный напиток']

In [190]:
assistant.process_message("давай что нибудь с рыбкой")

Текущие товары
['Святослав Пече Neill - безумец (970 г.)', 'Рол "Зеленый Дракон" (8 шт./250 г)', 'Картофельные чипсы куриные кари (50 г)', 'Салат из лосося (250 г)', 'Пицца Маргарита (400 г)', 'Coca-Cola (1 л)']
Обновленные товары
Рол "Зеленый Дракон" (8 шт./250 г)
Салат из лосося (250 г)
Свежий тунец в азиатском соусе (150 г)
Пицца Маргарита (400 г)
Coca-Cola (1 л)
Новый продукт
Свежий тунец в азиатском соусе (150 г)


In [191]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Рол "Зеленый Дракон" (8 шт./250 г) \n Нори, рис, угорь, огурец, томаго, икра масаго, авокадо, унаги соус, кунжут.',
 'Салат из лосося (250 г) \n Филе лосося слабосоленого, киноа, сыр филадельфия, авокадо, бобы эдамаме, помидоры черри, микс салата, заправка для салата, огурец, начос, кунжут',
 'Пицца Маргарита (400 г) \n Тесто, соус, сыр, помидор, моцарелла, зелень, пармезан.',
 'Coca-Cola (1 л) \n Сладкий газированный напиток',
 'Авторский ролл из туны (344 г) \n Тихоокеанский тунец, тигровая креветка в темпуре, перец чили, соус терияки, спелый авокадо, соус тонкоцу, микрозелень.']

In [169]:
assistant.process_message("замени картошку на спрайт")

['🔄 Анализирую ваш заказ 🍟', '💰 Ищу альтернативу картофелю 🥤', '📦 Проверяю наличие спрайта 🧃']

ОБЪЯСНЕНИЕ:
 Нашел для вас подходящий товар!  В ассортименте есть Bounty "Райское наслаждение" - отличный выбор напитка, который обладает высоким рейтингом 4.77 и вам полагается персональный кешбэк 20%! Хотите оформить заказ? 



In [170]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Сет "На двоих" \n Ролл "Филадельфия лайт", Ролл "филадельфия гриль", Ролл "сяке темпура"',
 'Bounty \n Райское наслаждение',
 'Пицца "Мексика" \n Халапеньо, фарш говяжий, соус томатный, сыр "Моцарелла", перец светофор']

In [174]:
assistant.process_message("замени bounty на кока колу")

['🔄 Анализирую ваш запрос 🤔', '🥤 Ищу варианты напитков  🍹', '🛒 Сравниваю цены и бренды 💰']

ОБЪЯСНЕНИЕ:
 Нашел для вас подходящие товары! Coca-Cola  имеет отличный рейтинг 4.9 и у вас на этот товар персональный кешбек 5%! Хотите оформить заказ? 



In [175]:
assistant.get_full_name_by_id(assistant.current_product_ids)

['Сет "На двоих" \n Ролл "Филадельфия лайт", Ролл "филадельфия гриль", Ролл "сяке темпура"',
 'Coca-Cola \n 0,5/1л',
 'Sprite (250 мл.) \n Стекло',
 'Пицца "Мексика" \n Халапеньо, фарш говяжий, соус томатный, сыр "Моцарелла", перец светофор']